In [1]:
import folium
import json
import requests

In [2]:
def hereTilesUrl(**kwdict):
    """
    Return a HERE map tiles URL, based on default values that can be 
    overwritten by kwdict.
    """
    params = dict(app_id     = "Your app id",
                  app_code   = "Your app code",
                  scheme     = 'reduced.day',
                  tilesize   = '256',
                  tileformat = 'png8',
                  lg         = 'rus',
                  x          = '{x}',
                  y          = '{y}',
                  z          = '{z}',)

    params.update(kwdict)

    url = ('https://2.base.maps.api.here.com/maptile/2.1/maptile/newest/{scheme}/'
           '{z}/{x}/{y}/{tilesize}/{tileformat}'
           '?lg={lg}&app_id={app_id}&app_code={app_code}'
            ).format(**params)
    return url

In [107]:
def getCustomLocations(loc,rad,layer):
    """
    Return all points for layer
    from Custom Location API.
    """
    params = {"app_id": "Your app id",
              "app_code": "Your app code",
              "proximity": "{0},{1}".format(loc,rad),
              "layer_ids": layer}
    
    url = 'https://cle.api.here.com/2/search/proximity.json'
    
    respond = requests.get(url, params=params)
    jdata = respond.json()
    points = list()
    for item in jdata["geometries"]:
        mpoint = item["geometry"]
        mpoint = str(str(mpoint.split('((')[1]).split('))')[0])
        mpoint = mpoint.split(' ')
        points.append([float(mpoint[1]),float(mpoint[0])])
    return points
    

In [110]:
# Create a HERE tiles
tiles_url = hereTilesUrl()

# Initialize the map object
map = folium.Map(location=[59.925658, 30.331779],  
               zoom_start=11, 
               tiles=tiles_url, 
               attr='here.com'
               )

# Get all locations for layer from Custom Locations API
locations = getCustomLocations('59.925658, 30.331779',10000,'TEST_LAYER')

# Just an icon
iurl = 'https://img.icons8.com/doodle/48/000000/building--v2.png'

# Add points to map
for i in locations:
    folium.Marker(i, icon=folium.features.CustomIcon(iurl, icon_size=(35,35))).add_to(map)

# Show the map    
map